Use this notebook to clean the data to remove duplicates and null values

In [4]:
import sys
import warnings
from datetime import date, datetime
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pymysql
from sqlalchemy import create_engine, text
import string
import secrets
import pyminizip

# Set up display options for Jupyter
from IPython.core.interactiveshell import InteractiveShell
get_ipython().ast_node_interactivity = "all"

warnings.filterwarnings('ignore')

# Add custom path
sys.path.append('~/Documents')

# Load data
df = pd.read_csv(r"C:\Users\payju\Downloads\PJN_notebook\2024\Feb\test.csv")

# Extract credentials
user = df['Cred'][0]      # PRIVATE
password = df['Cred'][1]  # PRIVATE
url = df['Cred'][2]       # PRIVATE
port = df['Cred'][3]      # PRIVATE

# Database connection
db = 'payjustnow'
fullUrl = f'mysql+pymysql://{user}:{password}@{url}:{port}/{db}'
sqlEngine = create_engine(fullUrl)
dbConnection = sqlEngine.connect()
print('Connected')

# Current date and time
now = datetime.now().strftime("%Y_%m_%d")


Connected


Reading the file

In [5]:
df = pd.read_csv(r"C:\Users\payju\Downloads\PJN_notebook\2024\adhoc\Final_model\input_data\HC_CREDICO_INPUT_20250506.txt", sep='|')
df
# df['IdNo'] = df['IdNo'].astype(str).str.zfill(13)
# print("Total numnber of records: ", df.shape[0])
# print("Number of unique ID Numbers: ",df['IdNo'].nunique())
# print("Number of duplicate ID Numbers: ", df[df['IdNo'].duplicated()]['IdNo'].nunique())
# # ------ Dropping duplicates ------
# df.drop_duplicates(subset=['IdNo'], keep='first', inplace=True)
# print("Number of duplicate ID Numbers after dropping duplicates: ", df[df['IdNo'].duplicated()]['IdNo'].nunique())

,IdNo,OrdNo,RatingScale,Score_Segment,Risk_Decision
0,6204030261012,25448194,25.0,Thinfile,Decline
1,5611060265088,25330040,3.0,PayingSeg,Accept
2,5411240838080,25567661,19.0,NonPayingSeg,Decline
3,6301030413088,25574071,13.0,PayingSeg,Accept
4,4802250637084,25488909,1.0,PayingSeg,Decline
...,...,...,...,...,...
24104,0509270785082,25581531,20.0,Thinfile,Accept
24105,9503020156083,25581674,25.0,NonPayingSeg,Accept
24106,9505120886089,25581684,18.0,PayingSeg,Accept
24107,9507085967086,25581689,24.0,NonPayingSeg,Accept


PJN Existing Customers

In [6]:
# PREPARING DATA FOR SQL 
df_id = df['IdNo']
df_id_str = df_id.astype(str).tolist()
df_id_str = ', '.join(f"'{x}'" for x in df_id_str)
print(df_id_str)

'6204030261012', '5611060265088', '5411240838080', '6301030413088', '4802250637084', '5303150714080', '6409050325080', '5511270693089', '5807070866085', '6503151256085', '5305210671085', '6504275471089', '5806040755089', '5404200331089', '5205050858083', '6304241030089', '6602040451087', '6307220557089', '5408290328087', '6802120406080', '5810160923089', '5010235648080', '5408020753083', '4709070195088', '5606150889086', '5202185635080', '7109095673085', '6601110834081', '5509100814081', '5602140715085', '6806060585084', '6310180587088', '4804250553089', '4510140116082', '6608180298089', '5602230761080', '6402200726083', '6103130788082', '6105250443083', '6511190395087', '5905160662081', '5602290658085', '5009300712086', '6706210595084', '6505190468084', '6902020243086', '6209220312087', '5503150323085', '6608210596080', '5003230414087', '5908150626082', '6606190330082', '6605180573081', '5301145333081', '6411180479084', '5511200241082', '6308075977083', '6407200523083', '6506270260086

In [7]:
# ------ CHECKING IF THE ID NUMBERS HAVE A PJN ACCOUNT ------

query = f'''
SELECT  u.id AS user_id,
        u.first_name,
        u.last_name,
        u.email,
        u.id_number,
        DATE(u.data_use_consent_at) AS signup_date,
        (u.maximum_amount/100) facility
FROM users u 
WHERE u.id_number IN ({df_id_str}) 
AND u.data_use_consent_at IS NOT NULL;
'''

# Execute the query and load data into a pandas DataFrame
users = pd.read_sql(text(query), dbConnection)
users

,user_id,first_name,last_name,email,id_number,signup_date,facility
0,19733,Somila,Magodla,magodlasomila5@gmail.com,9805041224083,2020-06-10,0.0
1,22175,SYBIL,Markham,relevantlooks101@gmail.com,5303150714080,2020-06-24,2500.0
2,23876,Zanele,Mabanga,zanelemabanga1@gmail.com,9910061056082,2020-07-03,0.0
3,25477,Luzuko Neo,Swartbooi,luzukoswartbooi@gmail.com,8702256191084,2020-07-12,0.0
4,30416,Mirriam Chibi,Ledwaba,mcledwaba64@gmail.com,6407200523083,2020-08-04,2500.0
...,...,...,...,...,...,...,...
3284,4209883,Edna Mamosa,Maele,mamosamaele58@gmail.com,9608100615088,2025-05-15,500.0
3285,4210998,Margaret,Mshlangu,margaretmahlangu18@gmail.com,7212210786087,2025-05-15,0.0
3286,4211437,Ntlamelle Alphonce,Malakoane,malakoanentlamelle47@gmail.com,8501295322084,2025-05-15,668.0
3287,4211601,Oratile,Motepe,oratilemotepe6@gmail.com,9909160603083,2025-05-15,500.0


In [8]:
df = df[~(df['IdNo'].isin(users['id_number']))]
df = df['IdNo'].to_frame()
df['IdNo'] = df['IdNo'].astype(str).str.zfill(13)
df.rename(columns={'IdNo': 'id_number'}, inplace=True)

In [9]:

# Generate a random 14-character password
characters = string.ascii_letters + string.digits + string.punctuation
password = ''.join(secrets.choice(characters) for i in range(14))

# Print the generated password
df.to_csv(f'input_data/test_{now}.csv', index=False)
print(f"Generated password: {password}")

# Define the file paths
file_to_zip = f'input_data/test_{now}.csv'  # The file you want to zip
zipped_file = f'input_data/test_{now}.zip'  # The name of the output zip file
compression_level = 5  # Choose a compression level (1 to 9)

# Create a password-protected zip file
pyminizip.compress(file_to_zip, None, zipped_file, password, compression_level)

Generated password: TJEp9N[7RPi?U<
